<a href="https://colab.research.google.com/github/CIS-522/course-content/blob/main/tutorials/W3_MLPs/W3_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS-522 Week 3 Part 1
# Multi-Layer Perceptrons (MLPs)

__Instructor__: Konrad Kording

__Content creators:__ Arash Ash

---
# Tutorial Objectives
In this tutorial, we delve deeper by using one of the most famous deep learning models of all!

MLPs are arguably one of the most tractable models that we can use to study deep learning fundamentals. Here we will learn why MLPs are: 

* similar to biological networks
* good at function approximation
* can evolve linearly in weights 
* the case of deep vs. wide
* dependant on transfer functions
* sensitive to initialization

In [ ]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = 'value' #@param {type:"string"}
my_pod = 'Select' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','astute-jellyfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion']


# Recap the experience from last week



In [ ]:
#@title Video: Discussing Week 2 - Linear DL
import time
try: t0;
except NameError: t0=time.time()

from IPython.display import YouTubeVideo

video = YouTubeVideo(id="xlYttP5C_LY", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

We focused on linear deep learning last week. We discussed about Artificial Neural Networks and saw how it works, dynamics of learning and finally properties of high dimensional spaces. You should now have some intuition about deep learning systems we will learn. We also dived into PyTorch and autograd which are tools that make our life easy. 

In [ ]:
# @title Slides
from IPython.display import HTML
HTML('<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vSPvHqDTmMq4GyQy6lieNEFxq4qz1SmqC2RNoeei3_niECH53zneh8jJVYOnBIdk0Uaz7y2b9DK8V1t/embed?start=false&loop=false&delayms=3000" frameborder="0" width="480" height="299" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>')


Meet with your pod for 10 minutes to discuss what you learned, what was clear, and what you hope to learn more about.

In [ ]:
#@markdown Tell us your thoughts about what you have learned.
my_w2_upshot = '' #@param {type:"string"}

# Question of the week
What functional forms are good or bad for representing complex functions?

[answers include differentiable, hierarchical, smooth, dynamical, nonlinear]

---
# Setup

In [ ]:
# imports
import random
import pathlib

import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid
from IPython.display import HTML, display

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dev, torch.get_num_threads()

In [ ]:
# @title Seeding for reproducibility
seed = 522
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_deterministic(True)
def seed_worker(worker_id):
    worker_seed = seed % (worker_id+1)
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
# @title Dataset download
%%capture
!rm -r AnimalFaces32x32/
!git clone https://github.com/arashash/AnimalFaces32x32
!rm -r afhq/
!unzip ./AnimalFaces32x32/afhq_32x32.zip 

In [ ]:
# @title Figure Settings
import ipywidgets as widgets
%matplotlib inline 
fig_w, fig_h = (8, 6)
plt.rcParams.update({'figure.figsize': (fig_w, fig_h)})
%config InlineBackend.figure_format = 'retina'
my_layout = widgets.Layout()

In [ ]:
# @title Helper functions
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis(False)
    plt.show()

def progress(epoch, loss, epochs=100):
    return HTML("""
        <label for="file">Training loss: {loss}</label>
        <progress
            value='{epoch}'
            max='{epochs}',
            style='width: 100%'
        >
            {epoch}
        </progress>
    """.format(loss=loss, epoch=epoch, epochs=epochs))

---
# Section 1: Neuron Physiology

In [ ]:
#@title Video: Overview and Integrate-and-Fire Neurons
try: t1;
except NameError: t1=time.time()

video = YouTubeVideo(id="exTzHGfEAvU", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

## Section 1.1: Leaky Integrate and Fire (LIF) Neuron
The basic idea of LIF neuron was proposed in 1907 by Louis Édouard Lapicque, long before we understood the electrophysiology of a neuron (see a translation of [Lapicque's paper](https://pubmed.ncbi.nlm.nih.gov/17968583/) ). More details of the model can be found in the book [**Theoretical neuroscience**](http://www.gatsby.ucl.ac.uk/~dayan/book/) by Peter Dayan and Laurence F. Abbott.

The model dynamic is defined with the following formula,

$$
\frac{d V}{d t}=\left\{\begin{array}{cc}
\frac{1}{C_{m}}\left(-\frac{V}{R_{m}}+I \right) & t>t_{r e s t} \\
0 & \text { otherwise }
\end{array}\right.
$$

Note that $-\frac{V}{R_{m}}$ is the leakage current. When $I$ is sufficiently strong such that $V$ reaches a certain threshold value $V_{\rm th}$, it momentarily spikes and then $V$ is reset to $V_{\rm reset}< V_{\rm th}$, and voltage stays at $V_{\rm reset}$ for $\tau_{\rm ref}$ ms, mimicking the refractoriness of the neuron during an action potential (note that $V_{\rm reset}$ and $\tau_{\rm ref}$ is assumed to be zero in the lecture):

\begin{eqnarray}
V(t)=V_{\rm reset} \text{  for } t\in(t_{\text{sp}}, t_{\text{sp}} + \tau_{\text{ref}}]
\end{eqnarray}

where $t_{\rm sp}$ is the spike time when $V(t)$ just exceeded $V_{\rm th}$.

Thus, the LIF model captures the facts that a neuron:
- performs spatial and temporal integration of synaptic inputs 
- generates a spike when the voltage reaches a certain threshold
- goes refractory during the action potential
- has a leaky membrane 

For in-depth content on computational models of neuron, follow the [NMA](https://www.neuromatchacademy.org/) Week 3 Day 1 material on Real Neurons and specifically this [Tutorial](https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/master/tutorials/W3D1_RealNeurons/W3D1_Tutorial1.ipynb).


## Exercise 1: Simulating an LIF Neuron

We now write Python code to calculate equation we saw above and simulate the LIF neuron dynamics. 

In the cell below is given a function for LIF neuron model with it's arguments described. Note that, `simulation_time_step` and `refactory_period` have the unit `msec`. Now, it's your turn to implement this simple mathematical model of a neuron:

In [ ]:
def run_LIF(I, T = 50, dt = 0.1, t_rest = 0, tau_ref = 4,
            Rm = 1, Cm = 10, Vth = 1, V_spike = 0.5):
  """
  Simulate the LIF dynamics with external input current

  Args:
    I          : input current (mA)
    T          : total time to simulate (msec)
    dt         : simulation time step (msec)
    t_rest     : initial refractory time
    tau_ref    : refractory period (msec)
    Rm         : resistance (kOhm)
    Cm         : capacitance (uF)
    Vth        : spike threshold (V)
    V_spike    : spike delta (V)

  Returns:
    time       : time points
    Vm         : membrane potentials
  """

  time = torch.arange(0, T+dt, dt) # [TO-DO]
  
  # potential (V) trace over time
  Vm = torch.zeros(len(time)) 

  # iterate over each time step
  for i, t in enumerate(time):
    if t > t_rest:
      Vm[i] = Vm[i-1] + 1/Cm*(-Vm[i-1]/Rm + I)  * dt # [TO-DO]
    if Vm[i] >= Vth:
      Vm[i] += V_spike # [TO-DO]
      t_rest = t + tau_ref # [TO-DO]
  return time, Vm

sim_time, Vm = run_LIF(1.5)
# plot membrane potential trace
plt.plot(sim_time, Vm)
plt.title('LIF Neuron Output')
plt.ylabel('Membrane Potential (V)')
plt.xlabel('Time (msec)')
plt.show()

# Section 1.2: Nonlinearity of LIF Neurons

In [ ]:
#@title Video: Are Integrate-and-Fire Neurons Linear?

video = YouTubeVideo(id="6IzHZB7xf34", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

## Interactive Demo: F-I Explorer for different $R_m$
We know that neurons communicate by modulating the spike count. Therefore it makes sense to characterize their spike count as a function of input current. This is called the neuron's input-output transfer function (so simply F-I curve). Let's plot the neuron's F-I curve and see how it changes with respect to the membrane resistance? 

In [ ]:
# @title

# @markdown Make sure you execute this cell to enable the widget!

@widgets.interact(Rm=widgets.FloatSlider(1., min=0.5, max=10., step=0.1, layout=my_layout))

def plot_IF_curve(Rm):
  T = 100 # total time to simulate (msec)
  dt = 1 # simulation time step (msec)
  Vth = 1 # spike threshold (V)
  Is = torch.linspace(0, 2, 10)
  spike_counts = []
  for I in Is:
    _, Vm = run_LIF(I, T = T, Vth = Vth, Rm=Rm)
    spike_counts += [torch.sum(Vm > Vth)]

  plt.plot(Is, spike_counts)
  plt.title('LIF Transfer Function (I/F Curve)')
  plt.ylabel('Spike count')
  plt.xlabel('I (mA)')
  plt.show()

---
# Section 2: The need for MLPs

In [ ]:
#@title Video: The XOR Problem

try: t2;
except NameError: t2=time.time()

video = YouTubeVideo(id="PERmPT1cOP0", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

Here we are exposed to a limitation of linear systems, namely in solving XOR problem. One cannot draw a straight line to separate the positive (1) and negative examples(0). 
The non liearity of multilayer percepton comes in handy to solve this problem. 
We will visualize this conept in Exercise 2. 

## Exercise 2: Solving XOR 

* Play with the widget and observe that you can not solve XOR
* Now add one hidden layer with three units, play with the widget, and set weights by hand to solve XOR perfectly.

For the second part, you should set the weights by clicking on the connections and either type the value or use the up and down keys to change it by one increment. You could also do the same for the biases by clicking on the tiny square to each neuron's bottom left.
Even though there are infinitely many solutions, a neat solution when $f(x)$ is ReLU: 

$$y = f(x_1)+f(x_2)-f((x_1+x_2))$$

In [ ]:
# @title XOR Exercise
from IPython.display import HTML
HTML('<iframe width="1020" height="660" src="https://playground.arashash.com/#activation=relu&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=&seed=0.91390&showTestData=false&discretize=false&percTrainData=90&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false" allowfullscreen></iframe>')

---
# Section 3: Universal Function Approximation Theorem

In [ ]:
#@title Video: Universal Approximation
try: t3;
except NameError: t3=time.time()

video = YouTubeVideo(id="XXXYxolMVdw", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

We saw previously that multilayer perceptrons can be used to solve the XOR problem. Building off this experiment a natural question that comes to mind is - can we approximate any function using multilayer perceptrons?

 We learned about Universal Approximation theorem. The intuition behind the theorem is that given a sufficient number of basis functions we can approximate any function sufficiently well. These basis functions are present in the hidden layer of MLPs.  

## Exercise 3: Function Approximation with ReLU
We learned that one hidden layer MLPs are enough to approximate any smooth function! Now let's manually fit a Sine function using ReLU activation. The idea is to set the weights iteratively so that the slope changes in the new sample's direction.

In [ ]:
N_test = 1000
x_test = torch.linspace(0, 2*np.pi, N_test).view(-1, 1)
y_test = torch.sin(x_test)
plt.plot(x_test, y_test, label='ground truth')

N_train = 10
x_train = torch.linspace(0, 2*np.pi, N_train).view(-1, 1)
y_train = torch.sin(x_train)

W1 = torch.ones(1, N_train)  # [TO-DO]
b1 = - x_train.view(N_train)  # [TO-DO]

W2 = torch.zeros(N_train, 1)  # [TO-DO]
prev_slope = 0
for i in range(N_train-1):
  delta_x = x_train[i+1] - x_train[i]  # [TO-DO]
  slope = (y_train[i+1] - y_train[i]) / delta_x  # [TO-DO]
  W2[i] = slope - prev_slope  # [TO-DO]
  prev_slope = slope

y_hat = torch.relu(b1 + x_test @ W1) @ W2  # [TO-DO]

plt.plot(x_test, y_hat, label='estimated')
plt.legend()
plt.xlabel('x')
plt.ylabel('y(x)')
plt.show()

# Section ?: Completeness proof sketch

In [ ]:
#@title Video: Making Multi-Layer Perceptrons
video = YouTubeVideo(id="bAhrg8Z8_r8", width=854, height=480, fs=1)
print("Video available at https://youtube.com/watch?v=" + video.id)

video

In the previous segment we implemented a function to approximate any smooth function using MLPs. We saw that using Lipschitz continuity we can prove that our approximation is mathematically correct. MLPs are fascinating but before we get into the details on how to design them familiarize yourself with some basic terminology of MLPs- layer, neuron, depth, widht, weight, bias and activation function. Armed with these ideas we can now begin to design a MLP given it's input, hidden and output size.




# MLPs in Pytorch

In [ ]:
#@title Video 5:

## Exercise 4: Implement a general-purpose MLP in Pytorch
The objective is to design an MLP with these properties:
* works with any input (1D, 2D, etc.)
* construct any number of given hidden layers using ModuleList
* use the same given activation function in all hidden layers

In [ ]:
class Net(nn.Module):
    def __init__(self, actv, num_inputs, hidden_units, num_outputs):
        super(Net, self).__init__()

        exec('self.actv = nn.%s'%actv)   # [TO-DO]

        self.layers = nn.ModuleList()
        for i in range(len(hidden_units)):
          next_num_inputs = hidden_units[i] 
          self.layers += [nn.Linear(num_inputs, next_num_inputs)]   # [TO-DO]
          num_inputs = next_num_inputs

        self.out = nn.Linear(num_inputs, num_outputs)

    def forward(self, x):
        # flattening
        x = x.view(x.shape[0], -1)   # [TO-DO]

        for layer in self.layers:
          x = self.actv(layer(x))  # [TO-DO]
        x = self.out(x) # [TO-DO]
        return x
# test it
Net(actv='LeakyReLU(0.1)',
    num_inputs = 2,
    hidden_units = [100, 10, 5],
    num_outputs = 1)

# ReLU in practice


In [ ]:
#@title Video 6:

## Exercise 5: Benchmark Various ReLU Implementation
Implement and benchmark at least three different ReLU implementations. Use `%timeit` with test number 10 and repeat number 3. Which then takes an average of 10 runs and repeats 3 times, and reports the lowest (best) average time.


In [ ]:
x = torch.rand((10000, 10000)).to(dev) - 0.5
print("Pytorch : ", end='')
%timeit -n10 -r3 torch.relu(x)
print('-----------------------------------------------')

print("First: ", end='')
%timeit -n10 -r3 torch.max(x, torch.zeros_like(x))   # [TO-DO]
print('-----------------------------------------------')

print("Second: ", end='')
%timeit -n10 -r3 x * (x > 0)   # [TO-DO]
print('-----------------------------------------------')

print("Third: ", end='')
%timeit -n10 -r3 x[x<0] = 0   # [TO-DO]
print('-----------------------------------------------')

print("Forth: ", end='')
%timeit -n10 -r3 (torch.abs(x) + x) / 2   # [TO-DO]

# Classification with MLPs
Two potential loss functions
## CrossEntropyLoss
This criterion expects a class index in the range $[0, C-1]$ as the target for each value of a $1D$ tensor of size minibatch. There are other optional parameters like class weights and class ignores. Check the documentation here for more detail. Then in the simplest case, it calculates this,

$$
\operatorname{loss}(x, \text { class })=-\log \left(\frac{\exp (x[\text { class }])}{\sum_{j} \exp (x[j])}\right)=-x[\text { class }]+\log \left(\sum_{j} \exp (x[j])\right)
$$

## MultiMarginLoss
The loss corresponding to class j is calculated as follows,
$$
l_j(x, y)=\sum_{j\neq y} \max (0, \operatorname{margin}-x[y]+x[j])
$$
Then it is averaged over all the class elements and all the mini-batch samples.

In [ ]:
#@title Video 7:

## Exercise 6: Simulate a Spiral Classification Dataset
Let's turn this fancy-looking equation into a classification dataset

$$
\begin{array}{c}
X_{k}(t)=t\left(\begin{array}{c}
\sin \left[\frac{2 \pi}{K}\left(2 t+k-1\right)\right]+\mathcal{N}\left(0, \sigma^{2}\right) \\
\cos \left[\frac{2 \pi}{K}\left(2 t+k-1\right)\right]+\mathcal{N}\left(0, \sigma^{2}\right) 
\end{array}\right)
\end{array}, \quad 0 \leq t \leq 1, \quad k=1, \ldots, K
$$

In [ ]:
K = 4
sigma = 0.4
N = 1000
t = torch.linspace(0, 1, N)
X = torch.zeros(K*N, 2)
y = torch.zeros(K*N)
for k in range(K):
  X[k*N:(k+1)*N, 0] = t*(torch.sin(2*np.pi/K*(2*t+k)) + sigma**2*torch.randn(N))   # [TO-DO]
  X[k*N:(k+1)*N, 1] = t*(torch.cos(2*np.pi/K*(2*t+k)) + sigma**2*torch.randn(N))   # [TO-DO]
  y[k*N:(k+1)*N] = k   # [TO-DO]

plt.scatter(X[:, 0], X[:, 1], c=y)
plt.plot()

# Training and Evaluation

In [ ]:
#@title Video 8:

## Exercise 7: Implement it for Spiral Dataset
Steps to follow: 
  * Dataset shuffle
  * Train/Test split
  * Dataloader definition
  * Training and Evaluation

In [ ]:
# Shuffling
shuffled_indeces = torch.randperm(K*N)   # [TO-DO]
X = X[shuffled_indeces]
y = y[shuffled_indeces]

# Test Train splitting
test_size = int(0.2*N)   # [TO-DO]
X_test = X[:test_size]
y_test = y[:test_size]
X_train = X[test_size:]
y_train = y[test_size:]
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test)
plt.plot()

And making a Pytorch data loader out of it

In [ ]:
batch_size = 128
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=batch_size,
                         shuffle=False, num_workers=0)

train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, drop_last=True,
                        shuffle=True, num_workers=0, worker_init_fn=seed_worker)

In [ ]:
def train_test_classification(net, criterion, optimizer,
                              train_loader, test_loader,
                              num_epochs=1, verbose=True, 
                              training_plot=False):
  if verbose:
    progress_bar = display(progress(0, 0, num_epochs), display_id=True)

  net.train()   # [TO-DO]
  training_losses = []
  for epoch in range(num_epochs):  # loop over the dataset multiple times
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs = inputs.to(dev).float()    # [TO-DO]
          labels = labels.to(dev).long()   # [TO-DO]

          # zero the parameter gradients
          optimizer.zero_grad()   # [TO-DO]

          # forward + backward + optimize
          outputs = net(inputs)   # [TO-DO]

          loss = criterion(outputs, labels)   # [TO-DO]
          loss.backward()   # [TO-DO]
          optimizer.step()   # [TO-DO]

          # print statistics
          if verbose:
            training_losses += [loss.item()]
            running_loss += loss.item()
            if i % 10 == 9:    # update every 10 mini-batches
                progress_bar.update(progress(epoch+1, running_loss / 10, num_epochs))
                running_loss = 0.0

  net.eval()   # [TO-DO]
  def test(data_loader):
    correct = 0
    total = 0
    for data in data_loader:
        inputs, labels = data
        inputs = inputs.to(dev).float()
        labels = labels.to(dev).long()

        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)   # [TO-DO]
        total += labels.size(0)   # [TO-DO]
        correct += (predicted == labels).sum().item()   # [TO-DO]

    acc = 100 * correct / total    # [TO-DO]
    return total, acc

  train_total, train_acc = test(train_loader)
  test_total, test_acc = test(test_loader)

  if verbose:
    print('Accuracy on the %d training samples: %0.2f %%' % (train_total, train_acc))
    print('Accuracy on the %d testing samples: %0.2f %%' % (test_total, test_acc))

  if training_plot:
    plt.plot(training_losses)
    plt.xlabel('Batch')
    plt.ylabel('Training loss')
    plt.show()
  
  return train_acc, test_acc

In [ ]:
net = Net('ReLU()', X_train.shape[1], [128], K).to(dev)    # [TO-DO] 
criterion = nn.CrossEntropyLoss()    # [TO-DO]
optimizer = optim.Adam(net.parameters(), lr=1e-3)
num_epochs = 100
_, _ = train_test_classification(net, criterion, optimizer, train_loader,
                                 test_loader, num_epochs=num_epochs,
                                 training_plot=True)

# Visualization

In [ ]:
#@title Video 9:

## Exercise 8: Implement decision map visualization

In [ ]:
def sample_grid(M=500, x_max = 2.0):
  ii, jj = torch.meshgrid(torch.linspace(-x_max, x_max,M),
                          torch.linspace(-x_max, x_max, M))
  X_all = torch.cat([ii.unsqueeze(-1),
                     jj.unsqueeze(-1)],
                     dim=-1).view(-1, 2)    # [TO-DO]
  return X_all

def plot_decision_map(X_all, y_pred, X_test, y_test, M=500, x_max = 2.0, eps = 1e-3):
  decision_map = torch.argmax(y_pred, dim=1)    # [TO-DO]

  for i in range(len(X_test)):
    indeces = (X_all[:, 0] - X_test[i, 0])**2 + (X_all[:, 1] - X_test[i, 1])**2 < eps    # [TO-DO]
    decision_map[indeces] = (K + y_test[i]).long()    # [TO-DO]

  decision_map = decision_map.view(M, M).cpu()
  plt.imshow(decision_map, extent=[-x_max, x_max, -x_max, x_max], cmap='jet')
  plt.plot()

X_all = sample_grid()
y_pred = net(X_all)
plot_decision_map(X_all, y_pred, X_test, y_test)